In [1]:
from scipy import sparse
import tensorflow as tf
import numpy as np
from keras import Input
from keras.models import Sequential
from keras.layers import Dense, Dropout
import json

# Loading datasets

In [2]:
y_test =np.load('y_test.npy')
y_train =np.load('y_train.npy')
x_train = sparse.load_npz('x_train.npz')
x_test = sparse.load_npz('x_test.npz')

In [3]:
np.mean(x_train.data)

0.0916627241782537

In [4]:
np.median(x_train.data)

0.0721792339873748

In [5]:
for i in range(x_train.data.shape[0]):
    x_train.data[i] = x_train.data[i]-0.07
for i in range(x_test.data.shape[0]):
    x_test.data[i] = x_test.data[i]-0.07

In [6]:
testAmount = 30000
x_test = x_test[:testAmount]
y_test = y_test[:testAmount]

In [7]:
x_train = x_train.tocoo()
indices = np.mat([x_train.row, x_train.col]).transpose()
x_train = tf.SparseTensor(indices, x_train.data, x_train.shape)

x_test = x_test.tocoo()
indices = np.mat([x_test.row, x_test.col]).transpose()
x_test = tf.SparseTensor(indices, x_test.data, x_test.shape)

# Overfitted Model

In [13]:
model = Sequential()

model.add(Input(shape=(47236,), sparse=True))

model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))

model.add(Dense(103, activation='sigmoid',))


model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=1e-7)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

score = model.fit(x_train, y_train, epochs=30, batch_size=40 ,validation_data=(x_test, y_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 200)               9447400   
_________________________________________________________________
dense_20 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_21 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_22 (Dense)             (None, 103)               20703     
Total params: 9,548,503
Trainable params: 9,548,503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30


C:\Users\jptl2\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_4/dense_19/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_4/dense_19/embedding_lookup_sparse/Reshape:0", shape=(None, 200), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_4/dense_19/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


579/579 [==============================] - 9s 15ms/step - loss: 14.7530 - acc: 0.1226 - val_loss: 14.9500 - val_acc: 0.1328
Epoch 2/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7523 - acc: 0.1490 - val_loss: 14.9493 - val_acc: 0.1530
Epoch 3/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7516 - acc: 0.1682 - val_loss: 14.9486 - val_acc: 0.1696
Epoch 4/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7509 - acc: 0.1822 - val_loss: 14.9478 - val_acc: 0.1814
Epoch 5/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7501 - acc: 0.1931 - val_loss: 14.9471 - val_acc: 0.1886
Epoch 6/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7494 - acc: 0.1998 - val_loss: 14.9463 - val_acc: 0.1934
Epoch 7/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7486 - acc: 0.2047 - val_loss: 14.9455 - val_acc: 0.1966
Epoch 8/30
579/579 [==============================] - 8s 14ms/step

In [14]:
json.dump(score.history, open("OverfittedModel", 'w'))

# Dropout Model

In [15]:
model = Sequential()

model.add(Input(shape=(47236,), sparse=True))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(103, activation='sigmoid',))


model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=1e-7)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

score = model.fit(x_train, y_train, epochs=30, batch_size=40 ,validation_data=(x_test, y_test))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 200)               9447400   
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 103)              

C:\Users\jptl2\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_5/dense_23/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_5/dense_23/embedding_lookup_sparse/Reshape:0", shape=(None, 200), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_5/dense_23/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


579/579 [==============================] - 9s 15ms/step - loss: 14.7543 - acc: 0.0276 - val_loss: 14.9512 - val_acc: 0.0470
Epoch 2/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7537 - acc: 0.0339 - val_loss: 14.9506 - val_acc: 0.0594
Epoch 3/30
579/579 [==============================] - 8s 15ms/step - loss: 14.7531 - acc: 0.0394 - val_loss: 14.9500 - val_acc: 0.0713
Epoch 4/30
579/579 [==============================] - 9s 15ms/step - loss: 14.7525 - acc: 0.0437 - val_loss: 14.9494 - val_acc: 0.0836
Epoch 5/30
579/579 [==============================] - 8s 15ms/step - loss: 14.7519 - acc: 0.0519 - val_loss: 14.9488 - val_acc: 0.0960
Epoch 6/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7513 - acc: 0.0603 - val_loss: 14.9482 - val_acc: 0.1085
Epoch 7/30
579/579 [==============================] - 8s 14ms/step - loss: 14.7506 - acc: 0.0658 - val_loss: 14.9476 - val_acc: 0.1192
Epoch 8/30
579/579 [==============================] - 8s 14ms/step

In [16]:
json.dump(score.history, open("DropoutModel", 'w'))